In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [80]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
Y_test= pd.read_csv('gender_submission.csv')
Y_test = Y_test.drop(['PassengerId'],axis=1)


In [81]:
train=train.drop(["Name","PassengerId","Cabin","Ticket"],axis=1)
test=test.drop(["Name","PassengerId","Cabin","Ticket"],axis=1)


In [82]:
#values in age and embarked missing
#use fill.na

# mean of the specific column
mean_value=train['Age'].mean()
train['Age'] = train['Age'].fillna(mean_value)
mean_value1=test['Age'].mean()
test['Age'] = test['Age'].fillna(mean_value1)




In [83]:
#S most common and  2 missing

train['Embarked'] = train['Embarked'].fillna('S')

test['Embarked'] = test['Embarked'].fillna('S')
train['Embarked'].describe()


count     891
unique      3
top         S
freq      646
Name: Embarked, dtype: object

In [84]:
genders = {"male": 0, "female": 1}
data = [train, test]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [85]:
station = {"S": 0, "C": 1, "Q": 2}
for dataset in data:
    dataset['Embarked']=dataset['Embarked'].map(station)

In [86]:
train = (train - train.mean())/(train.max()-train.min())

In [87]:

X_train= train.drop("Survived",axis=1)
Y_train= train["Survived"]


In [88]:
B=np.zeros(7)



In [89]:
alpha=0.001
iterations=1000
X_train.shape


(891, 7)

In [90]:
def sigmoid(X_train,B):
    result_sigmoid  = np.exp(X_train.dot(B))/(1+ np.exp(X_train.dot(B)))
    return  result_sigmoid



In [91]:
 
m=len(Y_train)

def cost_function(X_train,Y_train,B):
    h=sigmoid(X_train,B)
    a=np.log(h)
    b=np.log(1-h)
    c=Y_train
    d= (1-Y_train)
    cost_function=(-c*a - b*d)/m
    cost_function=cost_function.sum()/m
    return cost_function



In [92]:
def weightchange(X_train, Y_train, B, alpha):
        m =len(X_train)
        h = sigmoid(X_train,B)
        loss = h-Y_train
        gradient = X_train.T.dot(loss)/m
        B = B - alpha*gradient
        return B


In [93]:
#features;X_train ;(891,8)
#labels :Y_train; (891,)
#weights ;B ;(8,)


def train(X_train, Y_train, B, alpha):
    cost_history=[0]*iterations#
   
    for iteration in range(iterations):
        B = weightchange(X_train, Y_train, B, alpha)
        
        cost = cost_function(X_train, Y_train, B)
        cost_history.append(cost)   
             
        return B, cost_history


In [94]:
newB,cost_history= train(X_train, Y_train, B, alpha)
newB


Pclass     -0.000069
Sex         0.000126
Age        -0.000006
SibSp      -0.000002
Parch       0.000005
Fare        0.000012
Embarked    0.000017
dtype: float64

In [95]:
h=sigmoid(test,newB)
h.describe()

count    417.000000
mean       0.500041
std        0.000177
min        0.499913
25%        0.499944
50%        0.499979
75%        0.500046
max        0.501493
dtype: float64

In [96]:
Y_pred = sigmoid(test,newB)
Y_pred

0      0.499933
1      0.499935
2      0.499917
3      0.499937
4      0.499987
5      0.499957
6      0.499970
7      0.500018
8      0.499981
9      0.499991
10     0.499930
11     0.499998
12     0.500231
13     0.499957
14     0.500134
15     0.500052
16     0.499963
17     0.499945
18     0.499966
19     0.499944
20     0.500090
21     0.499947
22     0.500068
23     0.500145
24     0.500751
25     0.499923
26     0.500177
27     0.499943
28     0.500018
29     0.499975
         ...   
388    0.499951
389    0.500004
390    0.500234
391    0.500065
392    0.499994
393    0.499932
394    0.499975
395    0.500171
396    0.499945
397    0.500193
398    0.499942
399    0.499937
400    0.500473
401    0.499976
402    0.500169
403    0.500102
404    0.500011
405    0.499984
406    0.499965
407    0.500560
408    0.499970
409    0.500018
410    0.499970
411    0.500244
412    0.499965
413    0.499931
414    0.500295
415    0.499917
416    0.499931
417    0.499979
Length: 418, dtype: floa

In [97]:
def sort(Y_pred):
    return 1 if Y_pred >=0.5 else 0


    

In [98]:
Y_pred=Y_pred.map(sort)
type(Y_pred)



pandas.core.series.Series

In [99]:
#series to dataframe?

In [100]:
Y_pred.to_frame()

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,1
8,0
9,0


In [101]:
type(Y_test)
Y_test

,Survived
0,0
1,1
2,0
3,0
4,1
5,0
6,1
7,0
8,1
9,0


In [102]:
Y_pred.shape

(418,)

In [103]:
a=Y_pred.values
b=Y_test.values

a.reshape((418,1))
b.reshape((418,1))



array([[0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
    

In [104]:
m=len(Y_test)
count=0
for i in range(m):
    if a[i] == b[i]:
        count=count+1
count

285

In [107]:
count/418

0.6818181818181818

In [105]:
def compare(Y_pred,Y_test):
    return 1-(float(np.count_nonzero(diff))/len(diff))
compare(Y_pred,Y_test)

-418.0

In [106]:
np.count_nonzero(Y_pred)

169

In [70]:
np.count_nonzero(Y_test)

152

In [38]:
m=len(Y_test)

In [ ]:
#doubt
#vectorize,flatten